In [30]:
# Importamos bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime
import re
from sklearn.ensemble import IsolationForest

# Establecemos las columnas
colz=["IP","timestamp","status code","bytes sent","user agent"]

li=[]
c=0

# Abrimos el archivo
with open("logs/access.log") as f:
    while(True):
        if(c==100000):
            break
        if(c%10000==0):
            print(c) # Esto nose, yo lo borraria
        try:
            x=f.readline()
            if x==None:
                break
            log=x.split(" ")
            if(len(log)<12):
                continue
            #print([log[0],log[3].strip('['),log[5].strip('"'),log[7].strip('"'),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
            li.append([log[0],log[3].strip('['),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
        except Exception as e:
            pass
            #print(e)
            #print(log)
            #print(int(log[8]),int(log[9]))
            #print("skipped an entry")
        c+=1

# basic feature engineering
logs_df=pd.DataFrame(li,columns=colz)
logs_df["timestamp"]=pd.to_datetime(logs_df["timestamp"],format='%d/%b/%Y:%H:%M:%S')

#logs_df["url_length"]=logs_df["URL"].apply(len)
#logs_df["url_depth"]=logs_df["URL"].apply(lambda x : x.count('/'))
#logs_df['num_encoded_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'%[0-9A-Fa-f]{2}', x)))
#logs_df['num_special_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'[|,;]', x)))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [31]:
# Diccionario de status codes -> descripción
status_descriptions = {
    200: "OK",
    301: "Moved Permanently",
    302: "Found",
    304: "Not Modified",
    400: "Bad Request",
    403: "Forbidden",
    404: "Not Found",
    499: "Client Closed Request",
    500: "Internal Server Error",
    502: "Bad Gateway",
    504: "Gateway Timeout"
}

# Función para asignar tipo
def get_tipo(codigo):
    if 100 <= codigo < 200:
        return "Informativo"
    elif 200 <= codigo < 300:
        return "Exitoso"
    elif 300 <= codigo < 400:
        return "Redirección"
    elif 400 <= codigo < 500:
        return "Error Cliente"
    elif 500 <= codigo < 600:
        return "Error Servidor"
    else:
        return "Otro"

# DataFrame con los códigos únicos de status
estados = pd.DataFrame(logs_df["status code"].unique(), columns=["id_estadoserver"])

# Agregar descripción
estados["descripcion"] = estados["id_estadoserver"].map(status_descriptions)

# Agregar tipo
estados["tipo"] = estados["id_estadoserver"].apply(get_tipo)

# (Opcional) Ordenar por status_code
estados = estados.sort_values("id_estadoserver").reset_index(drop=True)


In [32]:
estados

,id_estadoserver,descripcion,tipo
0,200,OK,Exitoso
1,301,Moved Permanently,Redirección
2,302,Found,Redirección
3,304,Not Modified,Redirección
4,400,Bad Request,Error Cliente
5,403,Forbidden,Error Cliente
6,404,Not Found,Error Cliente
7,499,Client Closed Request,Error Cliente
8,500,Internal Server Error,Error Servidor
9,502,Bad Gateway,Error Servidor
